In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rom_tools import rom_manager

In [ ]:
rom = rom_manager.RomManager("../roms/sm_clean_2.smc", "../roms/sm_jank.smc")

In [ ]:
obj_names = rom.parse()

In [ ]:
for key,value in obj_names.items():
    print(key)
    print(value)

In [ ]:
enemy_list_92b3 = obj_names["room_header_0x792b3"].state_chooser.default.enemy_list.enemies

In [ ]:
for e_p in enemy_list_92b3:
    e = obj_names[e_p]
    print(e.x_pos/16, e.y_pos/16)
    print(e.x_pos//16, e.y_pos//16)

In [ ]:
import numpy as np

In [ ]:
def get_fixed_tiles(room_header):
    fixed_tiles = set()
    all_states = [s.state for s in room_header.state_chooser.conditions] + [room_header.state_chooser.default]
    shapes = [state.level_data.level_array.layer1.shape for state in all_states]
    # Hopefully all level datas have the same shape...
    assert len(set(shapes)) == 1
    for state in all_states:
        # Add doors
        layer1 = state.level_data.level_array.layer1
        it = np.nditer(layer1, flags=["multi_index", "refs_ok"])
        for tile in it:
            tile = tile.item()
            if tile.tile_type == 9:
                fixed_tiles.add(it.multi_index)
        # Add all enemy positions
        #TODO: Some enemies may take up more than one tile...
        for enemy in state.enemy_list.enemies:
            e_pos = (enemy.x_pos//16, enemy.y_pos//16)
            fixed_tiles.add(e_pos)
        # Add PLM positions
        for plm in state.plms.l:
            plm_pos = (plm.x_pos//16, plm.y_pos//16)
            fixed_tiles.add(plm_pos)
        # Add screens that are either fully solid or fully air
        for x in range(layer1.shape[0] // 16):
            for y in range(layer1.shape[1] // 16):
                screen_x = x * 16
                screen_y = y * 16
                screen = layer1[screen_x:screen_x+16,screen_y:screen_y+16]
                test_solid = np.vectorize(lambda x: x.tile_type == 8)
                #print(test_solid(screen))
                test_air = np.vectorize(lambda x: x.tile_type == 0)
                if np.all(test_solid(screen)) or np.all(test_air(screen)):
                    for x_sub in range(16):
                        for y_sub in range(16):
                            fixed_tiles.add((screen_x + x_sub, screen_y + y_sub))
    return fixed_tiles

In [ ]:
room_header = obj_names["room_header_0x791f8"]
sc = room_header.state_chooser
len(get_fixed_tiles(room_header))

In [ ]:
room_header.state_chooser.conditions[0].state.level_data is room_header.state_chooser.default.level_data

In [ ]:
import pickle

In [ ]:
with open("output/obj_names.pickle", "wb") as f:
    pickle.dump(obj_names, f)

In [ ]:
with open("output/obj_names.pickle", "rb") as f:
    obj_names2 = pickle.load(f)

In [ ]:
rom.compile(obj_names2)

In [ ]:
rom.save_and_close()

In [ ]:
room_header = obj_names2["room_header_0x797b5"]
state_chooser = room_header.state_chooser
state_choice = obj_names2[state_chooser.conditions[0]]
state = state_choice.state
print(state.level_data.address.as_snes_bytes(3))
print(state.list)
print(state.bytes)
print(state.bytes[0].size)
print(state.true_bytes)
print(state.bytes[0].resolve())
state.address

In [ ]:
from rom_tools.compress import decompress
from rom_tools.address import Address

In [ ]:
problem_bytes = rom.read_from_clean(Address(0x23FF02), 10000)

In [ ]:
problem_level, size = decompress.decompress_with_size(problem_bytes, debug=True)

In [ ]:
960 * 16 + 2

In [ ]:
len(problem_level)

In [ ]:
6144 * 2.5

In [ ]:
rom = rom_manager.RomManager("../roms/sm_asp_1.smc", "../roms/sm_foo.smc")
rom_clean = rom_manager.RomManager("../roms/sm_clean.smc", "../roms/sm_foo2")

In [ ]:
from rom_tools.compress import decompress
from rom_tools.address import Address

In [ ]:
a = Address(0x27322e)
raw_data = rom.read_from_clean(a, 10000)

In [ ]:
data = decompress.decompress(data)

In [ ]:
raw_data_clean = rom_clean.read_from_clean(a, 10000)

In [ ]:
data_clean = decompress.decompress(data_clean)

In [ ]:
len(data_clean)

In [ ]:
levelsize = int.from_bytes(data_clean[0:2], byteorder="little")
print(levelsize)
data_layer2 = data[int(2 + 1.5 * levelsize):]
data_clean_layer2 = data_clean[int(2 + 1.5 * levelsize):]

In [ ]:
len(data_clean_layer2)

In [ ]:
for i in range(len(data_layer2)):
    if data_layer2[i] != data_clean_layer2[i]:
        print(i, data_layer2[i], data_clean_layer2[i])

In [ ]:
from rom_tools.leveldata_utils import *

In [ ]:
bit_clean = bit_array_from_bytes(data_clean, Coord(48,48))

In [ ]:
re_bytes = bytes_from_bit_array(bit_clean)

In [ ]:
for i in range(len(re_bytes)):
    if data_clean[i] != re_bytes[i]:
        print(i, data_clean[i], re_bytes[i])

In [ ]:
rom.save_and_close()
rom_clean.save_and_close()